# intro to pandas

This file is a [Jupyter](https://jupyter.org/) notebook. The output that appears here was created by a Python kernel when this page was created. You can type the commands that appear in a notebook file like this one into your Python shell (or run them in a Python script) and expect to see the same results, assuming you have the dependencies installed.

We'll be taking a look at a **library** called [pandas](http://pandas.pydata.org/) which gives us some important basic functionality for handling datasets in Python.

If you're not sure what commands are available to you, note that (like the Unix command line) iPython supports **tab-completion**.

## 1. pandas basics

In order to use code in a library, we need to **import** it. This makes the library code accessible to Python by bringing it into [scope](https://docs.python.org/3.5/tutorial/classes.html#python-scopes-and-namespaces). The conventional way to import pandas is like this:

In [1]:
import pandas as pd

Once it's in scope we can access functions and objects that live in pandas by calling them with the `pd` prefix. This prefix specifies the pandas **namespace**, which is a map between pandas objects and the names we use to access them.

For example, we can access the pandas DataFrame object like this:

In [2]:
test_df = pd.DataFrame({'animal': ['cow', 'pig', 'chicken'], 'count': [3, 6, 12]})
test_df

,animal,count
0,cow,3
1,pig,6
2,chicken,12


The **DataFrame** is the fundamental object we'll use to manipulate datasets in pandas. A DataFrame can be constructed from a instance of `dict`. It stores values (of possibly different types) in rows and columns, like a relational table. It uses an **index** to keep track of its records; this is the left-most column of integers you see above, without a heading.

Creating a DataFrame explicitly from a `dict` as above can be useful, but frequently we'll  use another function that allows us to easily create a DataFrame from an input file:

In [3]:
HTS_FILE = '~/gits/gads_26/datasets/state_hts.tsv'
data = pd.read_csv(HTS_FILE, sep='\t')
data.head()

,state,peak,elev_ft
0,Alabama,Cheaha Mountain,2405
1,Alaska,Denali,20320
2,Arizona,Humphreys Peak,12633
3,Arkansas,Magazine Mountain,2753
4,California,Mount Whitney,14495


The `read_csv` function is the workhorse for loading data into pandas. Its first argument is a path to an input file, and its second ([keyword](http://sys-exit.blogspot.com/2013/07/python-positional-arguments-and-keyword.html)) argument specifies that our data is tab-delimited. The default behavior of `read_csv` is to use the first row of the input file as a header row.

The `head` function works like its Unix counterpart, printing the first few records of `data`. In this case the default number of rows is 5, but we can change this behavior by being explicit:

In [4]:
data.head(10)

,state,peak,elev_ft
0,Alabama,Cheaha Mountain,2405
1,Alaska,Denali,20320
2,Arizona,Humphreys Peak,12633
3,Arkansas,Magazine Mountain,2753
4,California,Mount Whitney,14495
5,Colorado,Mount Elbert,14433
6,Connecticut,Mount Frissell-South Slope,2372
7,Delaware,Ebright Azimuth,442
8,Florida,Britton Hill,345
9,Georgia,Brasstown Bald,4784


Let's look at some of panda's basic exploratory tools. We've already seen the `head` function. Here's another useful one:

In [5]:
data.shape

(50, 3)

The `shape` attribute is a tuple that contains the dimensions (rows, columns) of the DataFrame. Note that the syntax is `shape` and not `shape()`, since it's an attribute of the DataFrame object and not a method.

Another important tool is the `describe` method, which gives a summary of the numeric features in our dataset:

In [6]:
data.describe()

,elev_ft
count,50.000000
mean,6161.780000
std,5086.229574
min,345.000000
25%,2058.750000
50%,4588.500000
75%,10616.500000
max,20320.000000


The output is limited to the `elev_ft` column, since this is our only numeric feature. In addition to the count, mean, and standard deviation of the data we also get five important percentiles (0% = min, 25% = first quartile, 50% = median, 75% = third quartile, 100% = max).

These percentiles comprise a **five-number summary** of the distribution of `elev_ft`. The five-number summary is a useful first approximation to the shape of the distribution of the data. It gives us a rough picture of central tendency, central variation, skew, and tail behavior.

This five-number summary suggests that the distribution of `elev_ft` is skewed and fat-tailed.

## 2. selecting data

Sometimes we'll want to use only a subset of our data at once. There are [several ways](http://pandas.pydata.org/pandas-docs/stable/indexing.html) to perform these kinds of selection operations on a DataFrame.

We can access a single column using the same syntax we use to access elements in a `dict`: 

In [7]:
data['state'].head()

0       Alabama
1        Alaska
2       Arizona
3      Arkansas
4    California
Name: state, dtype: object

We can also read columns using attribute notation (note this doesn't work when trying to write to a column):

In [8]:
data.state.head()

0       Alabama
1        Alaska
2       Arizona
3      Arkansas
4    California
Name: state, dtype: object

Let's take a look at our column's data type:

In [9]:
type(data.state)

pandas.core.series.Series

The column is stored as a Series, another fundamental data storage object in pandas. For our purposes, we will mostly see Series objects as constituent parts of a DataFrame.

Series objects have methods too, for example we can find the average height of our 50 highest peaks:

In [10]:
data.elev_ft.mean()

6161.7799999999997

This agrees with the output from `describe`.

Another useful way to select data is with a **boolean mask**. This is just a fancy term for an array of boolean (T/F) values that indicates which values to return:

In [11]:
data[data.state == 'California']

,state,peak,elev_ft
4,California,Mount Whitney,14495


Under the hood, the boolean condition we use here is an array of 50 T/F values, where the only T occurs at index 4.

We can access specific cells in the DataFrame using the `iloc` syntax. The `iloc` syntax is flexible and can take many different types of inputs (ints, arrays of ints, slice objects, [among others](http://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-position)).

Keep in mind when using `iloc` that the first argument specifies rows, the second argument specifies columns, and the arguments are separated by a comma. For example, the following command returns values in column `0` for rows `10-14` (like elsewere in Python, [slice objects](https://docs.python.org/3.5/tutorial/introduction.html) are lower index inclusive & upper index exclusive):

In [12]:
data.iloc[10:15, 0]

10      Hawaii
11       Idaho
12    Illinois
13     Indiana
14        Iowa
Name: state, dtype: object

Omitting the second argument returns all the columns for these rows:

In [13]:
data.iloc[10:15]

,state,peak,elev_ft
10,Hawaii,Mauna Kea,13796
11,Idaho,Borah Peak,12662
12,Illinois,Charles Mound,1235
13,Indiana,Hoosier Hill,1257
14,Iowa,Hawkeye Point,1670
